In [1]:
import torch
import numpy as np
import cv2
import time
import datetime
import pandas as pd
from PIL import Image
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression

from convert_data import collect_data
from class_mapping import yolo_class_mapping, fire_class_mapping, animals_class_mapping
from detection import category

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



#### Version check

In [2]:
print('opencv version: ', cv2.__version__)

opencv version:  4.8.0


In [3]:
camera_name = 'KFQ_StarValley'

# Load trained YOLOv5 model weights
yolo_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/yolov5m.pt'
device = torch.device('cpu')
yolo_model = attempt_load(yolo_weights_path, device=device)

fire_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/fire_smoke.pt'
fire_model = attempt_load(fire_weights_path, device=device)

animals_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/animals.pt'
animals_model = attempt_load(animals_weights_path, device=device)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create an empty DataFrame to store results
columns = ['Time', 'Class_Name', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'action_detection', 'action_category']
results_df = pd.DataFrame(columns=columns)

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    captured_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 상황 발생 시간 기록
    
    # Preprocess the frame
    image = Image.fromarray(frame)
    image = image.resize((640, 640))  # Resize the image to the model's input size
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # Perform object detection
    results = yolo_model.forward(image_tensor)[0]
    results = non_max_suppression(results, conf_thres=0.5)
    results_df = collect_data(results, frame, yolo_class_mapping, results_df, captured_time)
    print(results_df)
    
    results = fire_model.forward(image_tensor)[0]
    results = non_max_suppression(results, conf_thres=0.5)
    results_df = collect_data(results, frame, fire_class_mapping, results_df, captured_time)
    print(results_df)
    
    results = animals_model.forward(image_tensor)[0]
    results = non_max_suppression(results, conf_thres=0.5)
    results_df = collect_data(results, frame, animals_class_mapping, results_df, captured_time)
    print(results_df)
    
    results_df = category(results_df)

    # Display the frame with bounding boxes and labels
    cv2.imshow(camera_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
        break
    
    time.sleep(2) # 2초 간격으로 사진 캡쳐(30 frames per 1 miniute)

# Save the DataFrame to a CSV file
results_df.to_csv('object_detection_results.csv', index=False)

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Fusing layers... 
YOLOv5m summary: 369 layers, 21190557 parameters, 0 gradients
Fusing layers... 
YOLOv5m summary: 369 layers, 20875359 parameters, 0 gradients, 48.2 GFLOPs
Fusing layers... 
YOLOv5m summary: 369 layers, 20887482 parameters, 0 gradients, 48.3 GFLOPs


Empty DataFrame
Columns: [Time, Class_Name, Confidence, X1, Y1, X2, Y2, action_detection, action_category]
Index: []
Empty DataFrame
Columns: [Time, Class_Name, Confidence, X1, Y1, X2, Y2, action_detection, action_category]
Index: []
Empty DataFrame
Columns: [Time, Class_Name, Confidence, X1, Y1, X2, Y2, action_detection, action_category]
Index: []
                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399  1031   

   action_detection action_category  
0                 0             NaN  
                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399  1031   
1  2023-08-17 12:12:57      smoke  tensor(0.54547)   191  190   714   407   
2  2023-08-17 12:12:57       fire  tensor(0.53792)   758  246   833   302   

   action_detection action_category  
0                 0             NaN  
1                 0             N

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['actio

                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399  1031   
1  2023-08-17 12:12:57      smoke  tensor(0.54547)   191  190   714   407   
2  2023-08-17 12:12:57       fire  tensor(0.53792)   758  246   833   302   
3  2023-08-17 12:12:57       deer  tensor(0.58626)   391  928   646  1078   
4  2023-08-17 12:12:57       deer  tensor(0.55084)     1  912   152  1082   
5  2023-08-17 12:13:00     person  tensor(0.74394)  1210  681  1367   973   

   action_detection        action_category  
0                 1        person detected  
1                 0                    NaN  
2                 0                    NaN  
3                 1  animal detected(deer)  
4                 1  animal detected(deer)  
5                 0                    NaN  
                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399 

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['actio

                   Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0   2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399  1031   
1   2023-08-17 12:12:57      smoke  tensor(0.54547)   191  190   714   407   
2   2023-08-17 12:12:57       fire  tensor(0.53792)   758  246   833   302   
3   2023-08-17 12:12:57       deer  tensor(0.58626)   391  928   646  1078   
4   2023-08-17 12:12:57       deer  tensor(0.55084)     1  912   152  1082   
5   2023-08-17 12:13:00     person  tensor(0.74394)  1210  681  1367   973   
6   2023-08-17 12:13:00       fire  tensor(0.58778)   730  192   799   249   
7   2023-08-17 12:13:00      smoke  tensor(0.56362)   143  126   673   349   
8   2023-08-17 12:13:00       deer  tensor(0.57142)   269  883   615  1078   
9   2023-08-17 12:13:02     person  tensor(0.81118)  1177  602  1348   903   
10  2023-08-17 12:13:02     person  tensor(0.60485)  1519  580  1607   679   

    action_detection        action_category  
0                

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['actio

                   Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0   2023-08-17 12:12:57     person  tensor(0.69456)  1242  732  1399  1031   
1   2023-08-17 12:12:57      smoke  tensor(0.54547)   191  190   714   407   
2   2023-08-17 12:12:57       fire  tensor(0.53792)   758  246   833   302   
3   2023-08-17 12:12:57       deer  tensor(0.58626)   391  928   646  1078   
4   2023-08-17 12:12:57       deer  tensor(0.55084)     1  912   152  1082   
5   2023-08-17 12:13:00     person  tensor(0.74394)  1210  681  1367   973   
6   2023-08-17 12:13:00       fire  tensor(0.58778)   730  192   799   249   
7   2023-08-17 12:13:00      smoke  tensor(0.56362)   143  126   673   349   
8   2023-08-17 12:13:00       deer  tensor(0.57142)   269  883   615  1078   
9   2023-08-17 12:13:02     person  tensor(0.81118)  1177  602  1348   903   
10  2023-08-17 12:13:02     person  tensor(0.60485)  1519  580  1607   679   
11  2023-08-17 12:13:02       fire  tensor(0.58497)   686  113  

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['actio